# CNN for Urban Region

In [1]:
import os
from zipfile import ZipFile
import pandas as pd
import numpy as np
import rasterio
import random
import tensorflow
from sklearn.model_selection import train_test_split
import shutil
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

#to dos:
#generalize some code like pixel counts
#create a preprocessing script and outsource methods
#write different methods for normalization
#write some quality control methods, e.g. pdf (probability density functions, outlyer detection - implement
#into normalization methods)
#more to come

In [2]:
print("Num CPUs Available: ", len(tf.config.list_physical_devices("CPU")))
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Num CPUs Available:  1
Num GPUs Available:  0


2021-09-13 09:57:33.472079: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:57:33.473297: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:57:33.474457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-13 09:57:33.480231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-09-13 09:57:33.480240: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries.

In [3]:
#Move Urban images to own directory urban
def get_urban(urban_dir:str, orig_dir: str, channels: int):
    
    #List all images
    img_list = os.listdir(orig_dir)
    survey_name = os.path.basename(orig_dir)
    
    channel_size = len(channels)
    
    for img_name in img_list:
        if img_name.endswith('.tif'):
            img_dir = os.path.join(orig_dir, img_name)
            with rasterio.open(img_dir) as img:
                if len(channels) == 0:
                    try:
                        array = img.read()
                        channel_size = array.shape[0]
                    except Exception as e:
                        print(e, 'img_dir is', img_dir)
                        continue
                else:
                    try:
                        array = img.read(channels)
                        channel_size = array.shape[0]
                    except Exception as e:
                        print(e, 'img_dir is', img_dir)
                        continue
            if array.shape[1]< 500:
                #Name tif data survey name + cluster
                #img_survey_name = img_name.replace(img_name[:6], survey_name)
                urb_img = os.path.join(urban_dir, img_name)
                os.rename(img_dir, urb_img)
                
    
    return channel_size

#Extract from each zip-file (each survey) all images which belongs to urban category (less than 500x500 pixels) and
#move them to own directory
def get_urban_img(img_zip_dir: str,base: str, channels: int):
    
    channel_size = 0
    
    #Create Urban Folder if not existing
    urban_dir = os.path.join(base, 'urban')
    if not os.path.exists(urban_dir):
        os.mkdir(urban_dir)

        
    zip_files = os.listdir(img_zip_dir)
    #Extract for each survey zip file all files into a temporary directory
    for zip_file in zip_files:
        
        if zip_file.endswith('.zip'):
            #Current zip file directory
            zip_dir = os.path.join(img_zip_dir, zip_file)
            #Unzip all files to img_dir (temporary directory)
            with ZipFile(zip_dir, 'r') as zipObj:
                
                img_dir_name = os.path.splitext(zip_file)[0]
                img_dir = os.path.join(base, img_dir_name)
                # Extract all the contents of zip file into temporary directory 
                zipObj.extractall(img_dir)
                # Jump into get_urban to move all urban images to the urban folder; return channel size
                channels_num = get_urban(urban_dir, img_dir, channels)
                #If channel size is not zero replace channel size with new value
                if channels_num != 0:
                    channel_size = channels_num
                #Delete temporary directory
                shutil.rmtree(img_dir)

    return urban_dir, channel_size
    


In [4]:
#Create dataframe where the column source refers to the water source which was named the most

#Get the index of the last position of surveyname
def get_pos_last_letter(cluster_name:str):
    
    #Reverse string
    string = cluster_name[::-1]
    #Check for first non-digit entry
    for i in string:
        if i.isalpha():
            #Get position in reversed string
            pos_reversed = string.find(i)
            # 'Translate' Position of reversed string for non-reversed string
            pos = len(cluster_name)-pos_reversed-1
            break
            
    return pos

def get_cluster_num_from_img_name(img:str):
    
    cluster_num = None
    #Remove .tif (BFGE71FL0000203.tif )
    img = img[:img.rfind('.')]
    
    #Remove survey name (alphabetich letter, BFGE71FL0000203)
    #Check where survey name ends by finding last alphabetic letter
    pos_last_alpha = get_pos_last_letter(img)
    #Get cluster (including zeros) (0000203)
    cluster_0 = img[pos_last_alpha+1:]
    
    #Get cluster number by finding first entry in remaining string which is not 0 as they typically have the form
    #0000203 (want 203)
    num_start = None
    for str_index in range(0, len(cluster_0)):
        if cluster_0[str_index] != '0':
            num_start = str_index
            break
    if num_start is None:
        cluster_num = 0
    else:
        cluster_num = cluster_0[num_start:]
            
    return cluster_num
    
    
def get_main_source_file(water_file:pd.DataFrame):
   
    #Remove unnamed column
    water_file = water_file.loc[:, ~water_file.columns.str.contains('^Unnamed')]
    #Remove all unnecessary columns for our purpose
    water_source = water_file.drop(labels=['ID','cluster', 'residence','year'], axis = 1)
    #replace Na with 0 for the next step idxmax
    water_source = water_source.fillna(0)
    #Get a "df" where for each cluster the most dominating water source is given
    water_source=  water_source.idxmax(axis=1)
    #Naming of the column with most dominating water source
    water_source.name = 'source'

    #Merge, columns with ID and cluster of original df with the 'df' (which has the dominating water source)
    df = pd.concat([water_file['ID'], water_file['cluster'], water_source], axis = 1)
    
    return df

#Create label dataframe where next to name (image name (Survey name+Cluster number)) the label in string format
#and categorial format (number) is provided, but only for sentinel images which main source belongs to the main label
#defined prior (which we want to classify); if they don't belong to one of those main labels thoses images are removed
def get_labels_df_for_img(csv_file:str, urban_dir:str, main_labels:list, corr_GE_HR_file:str):
    
    #List all images in urban directory
    img_list = os.listdir(urban_dir)
    #Read in the Water file CSV file 
    water_file = pd.read_csv(csv_file)
    #Create empty list for the labels (used as temporary storage before transforming it int a dataframe)
    label_list = []
    #Define column_names for the final data frame 
    column_names =  ["name", "source", "label"]
    #Jump into function to get water source data frame where a column for the most dominating source (value is a
    #string indicating the most dominating source for each cluster replaces the columns of possible water source and
    #the count how often they were named by a cluster)
    water_file_max = get_main_source_file(water_file)
    #Read in file which connects Geodata files with HR data files
    corr_ge_hr = pd.read_csv(corr_GE_HR_file)

    #Iterate over all images
    for img in img_list:
        #no_label -> for kicking out image who dont have a label
        no_label = True
        #Iterate over all rows of the water_file_max data frame
        for index, survey_name in enumerate(water_file_max['ID']): 
                #Get corresponding GeoData file name for HR file survey name via the df corr_ge_hr
                idx = corr_ge_hr.index[corr_ge_hr['HR'] == survey_name]
                ge_name = corr_ge_hr.iloc[idx[0]]['GE']
                #Check if GeoData file name is in the image name (only survey name, not cluster checked yet)
                if ge_name in img:
                    #Get cluster number from image; note +.tif to ensure that e.g. 1 is not true for e.g. AOGe....2104.tif
                    cluster_img = int(get_cluster_num_from_img_name(img))
                    #Get cluster number from possible fitting water source df row
                    cluster_df = int(water_file_max.loc[index]['cluster'])
                    #check if the cluster numbers are identical
                    if cluster_img == cluster_df:
                        no_label = False
                        #Check if label is part of the main labels (if not remove the image)
                        #Get water source
                        source = water_file_max.loc[index]['source']
                        #Check if source in main labels (predefined parameter)
                        #If yes, add current image name, and the water source twice (one will be turned to a numerical category)
                        if source in main_labels:
                            img_label = [img, source, source]
                            label_list.append(img_label)
                            print('Kept', img, survey_name, cluster_df, source)
                            break
                        else:
                            #If not,delete image from the directory urban
                            img_dir = os.path.join(urban_dir,img)
                            os.remove(img_dir)
                            print('Deleted',img, survey_name, cluster_df, source)
                            break
        if no_label == True:
            #If no corresponding label was found delete from urban_dir
            img_dir = os.path.join(urban_dir,img)
            os.remove(img_dir)
            print('Removed as no label',img)
                            
                            
    #Get data frame for label list                        
    label_array = np.array(label_list)
    label_df = pd.DataFrame(label_array, columns = column_names) 
    #Turn label column (second water source) from string entries to categorical entries
    label_df.label = pd.Categorical(pd.factorize(label_df.label)[0])

    return label_df

In [5]:
#Split data set into training, validation, and testing; each subset gets its own folder

#Get ratio between validation to test set to be able to split the remaining data set properly (prior: Split into
#training set and remaining set)
def ratio_val_to_test(val:float, test:float):
    
    total = val + test
    one_perc = 100.00/total
    val_ratio = one_perc * val*0.01
    
    return val_ratio

def created_data_sets(urban_dir:str, split_size):

    print('split size', type(split_size))
    img_list = os.listdir(urban_dir)

    #Create  validation, training und test folder

    train_dir = os.path.join(urban_dir,'training')
    if not os.path.exists(train_dir):
        os.mkdir(train_dir)

    val_dir = os.path.join(urban_dir,'validation')
    if not os.path.exists(val_dir):
        os.mkdir(val_dir)

    test_dir = os.path.join(urban_dir,'test')
    if not os.path.exists(test_dir):
        os.mkdir(test_dir)

    #Split into the data sets and move them to their respective folder
    
    X_train, X_rem = train_test_split(img_list, train_size= split_size[0])
    
    X_val, X_test = train_test_split(X_rem, train_size = ratio_val_to_test(split_size[1], split_size[2]))

    for img in X_train:
            img_dir = os.path.join(urban_dir, img)
            train_img = os.path.join(train_dir, img)
            os.rename(img_dir, train_img)

    for img in X_val:
            img_dir = os.path.join(urban_dir, img)
            val_img = os.path.join(val_dir, img)
            os.rename(img_dir, val_img)

    for img in X_test:
            img_dir = os.path.join(urban_dir, img)
            test_img = os.path.join(test_dir, img)
            os.rename(img_dir, test_img)
    
    
    return train_dir, val_dir, test_dir

In [6]:
#Alternative calculation way of mean and std if dataset is not too big
def calc_mean_std(data_dir:str, input_height, input_width,clipping_values, channels, channel_size):
    
    img_list = os.listdir(data_dir)
    
    assert all([i.endswith('.tif') for i in img_list])
    
    pixels = np.ndarray(shape(len(img_list), channel_size, input_height, input_width))
    
    #Create "array of all images"
    for i, img_name in enumerate(img_list):
        img_dir = os.path.join(data_dir, img_name)
        with rasterio.open(img_dir) as img:
            if len(channels) == 0:
                array = img.read()
            else:
                array = img.read(channels)

            
        array = array.astype('float32')
        #Clipping
        array = np.clip(array,a_min = clipping_values[0], a_max = clipping_values[1])
        #Ensure that that all arrays have the same size
        array = array[:,:input_height,:input_width]
        pixels[i] = array
        
    #Calculate Mean and Standard deviation along images (axis 0), width & heigth(axis:2,3) for each channel (axis:1)       
    means = pixels.mean(axis=(0,2,3), dtype='float64')
    stds = pixels.std(axis=(0,2,3), dtype='float64')

    
    return means, stds

In [7]:
#Calculate mean for each channel over all pixels for training set; for validation and test set you need to take
#mean and std of training set as well as in real case scenarios you don't know them beforehand to calculate them
def calc_mean(data_dir:str,input_height, input_width, clipping_values, channels):
    
    img_list = os.listdir(data_dir)
    
    #Ensures that only tif data are in the current directory, if not it will throw an error
    assert all([i.endswith('.tif') for i in img_list])
    
    #Variable to save the summation of the pixels values
    sum_arr = 0
    #Count of pixels
    sum_pixel = input_height*input_width*len(img_list)

    for i, img_name in enumerate(img_list):
        img_dir = os.path.join(data_dir, img_name)
        with rasterio.open(img_dir) as img:
            #Read in image with all channels
            if len(channels) == 0:
                array = img.read()
            #Only read defined channels of the image
            else:
                array = img.read(channels)
        
        array = array.astype('float32')
        #Replace NaN with zeros (for calculation required)
        array[np.isnan(array)] = 0
        #Clipping
        array = np.clip(array,a_min = clipping_values[0],a_max = clipping_values[1])
        #Ensure that that all arrays have the same size
        array = array[:,:input_height,:input_width]
        #Adding the sum over the input and height for each channel seperately up
        sum_arr += array.sum(axis = (1,2))
        
    #Calculate mean for each channel
    means = sum_arr/sum_pixel
    
    #return channelwise mean (#mean == # channels)
    return means

In [8]:
#Calculate standard deviation (note:mean function has to be executed beforehand as it is required as input)
def calc_std(means, data_dir, input_height, input_width, clipping_values, channels):
    
    img_list = os.listdir(data_dir)
    #Ensure all data are tif data in current directory
    assert all([i.endswith('.tif') for i in img_list])
    
    sum_arr = 0
    #Count of pixels
    sum_pixel = input_height*input_width*len(img_list)
    
    # Sum each array/image up (channelwise) with each other, after substracting mean(s) from each array and take it ^2
    #each one seperately -> Check standard deviatin equation
    for i, img_name in enumerate(img_list):
        
        img_dir = os.path.join(data_dir, img_name)
        with rasterio.open(img_dir) as img:
            if len(channels) == 0:
                #Read in all channels
                array = img.read()
            else:
                #Only read in predefined channels
                array = img.read(channels)

        array = array.astype('float32')
        array[np.isnan(array)] = 0

        #Clipping
        array = np.clip(array,a_min = clipping_values[0],a_max =clipping_values[1])
        #Ensure that that all arrays have the same size
        array = array[:,:input_height,:input_width]
    
        array = np.power(array.transpose(1,2,0) - means, 2).transpose(2, 0, 1)
        sum_arr += array.sum(axis = (1,2))
    
    #Second part of equation
    stds = np.sqrt(sum_arr/sum_pixel)
    
    return stds

In [9]:
#Create class weights for the prediction model as our data set is imbalanced (acoount higher weight to classes with 
#less samples)
def create_class_weights_dict(train_dir:str, labels_df:str):
    
    train_sample  = os.listdir(train_dir)
    label_array = np.zeros(len(train_sample), dtype=int)
    index = 0

    #Create array with the labels used (training labels) such that we get a list of the form [1 1 1 0 2 1 2....]
    for sample in train_sample:
        got = False
        for pos, survey_name in enumerate(labels_df['name']):
            if survey_name in sample:
                label = labels_df.loc[pos]['label']
                label_array[index] = label
                index += 1
                got = True
        if got == False:
            print(sample)
     
    class_weights = class_weight.compute_class_weight('balanced',np.unique(label_array), label_array)
    class_weights_dict = dict(enumerate(class_weights,))
    
    return class_weights_dict

    

In [10]:
#Generate iterable object for model.fit()
def generator(x_dir, labels, batch_size, means, stds, input_height, input_width, clipping_values, channels, channel_size, num_labels):

    x_list = os.listdir(x_dir)
    assert all([i.endswith('.tif') for i in x_list])
    #Shuffle elements in list, so that batches consists of images of different surveys
    random.shuffle(x_list)
    #generate batches (x : input, y: label)
    batch_x = np.zeros(shape=(batch_size, channel_size,input_height, input_width))
    batch_y = np.zeros(shape=(batch_size,num_labels), dtype=int)
    #Iterator
    batch_ele = 0

    for x in x_list:
        #Get training sample x
        img_dir = os.path.join(x_dir, x)
        
        with rasterio.open(img_dir) as img:
            #if we want to use all channels
            if len(channels) == 0:
                array = img.read().astype("float32")
            else:
                array = img.read(channels).astype("float32")

        
        array[np.isnan(array)] = 0
        assert not np.any(np.isnan(array)), "Float"
        #Clipping
        array = np.clip(array,a_min = clipping_values[0],a_max = clipping_values[1])

        assert not np.any(np.isnan(array)), "After clipping"
        #Ensure that that all arrays have the same size via cropping
        array = array[:,:input_height,:input_width]
        
        #Normalize the array
        array = ((array.transpose(1,2,0)-means)/stds).transpose(2, 0, 1)
        assert not np.any(np.isnan(array)), "Normalize"
        # Add to batch
        batch_x[batch_ele] = array     
        
        #Get corresponding label y
        for index, survey_name in enumerate(labels['name']):
            if survey_name in x:
                one_hot = np.zeros(shape = num_labels)
                label_pos = (labels.loc[index]['label'])
                #One hot encoding
                one_hot[label_pos] = 1
                batch_y[batch_ele] = one_hot
                
        #Check if batch is already full (Note: Index in batch array is from 0...4 hence we need to add +1 to batch_ele)
        if (batch_ele+1) == batch_size:
            batch_x = batch_x.transpose(0,2,3,1)
            #Return of batch_x,batch_y
            yield batch_x.astype(np.float32), batch_y.astype(np.float32)
            #Reset settings -> Start of next batch generation
            batch_ele = 0
            batch_x = np.zeros(shape=(batch_size, channel_size,input_height, input_width))
            batch_y = np.zeros(shape=(batch_size, num_labels), dtype=int)

        else:
            batch_ele += 1
    
    
#    return batch_x, batch_y

In [11]:
#PARAMETERS

#base where Script and CSV File about water sources are stored
base = '/mnt/datadisk/shannon/NN/'
img_dir = '/mnt/datadisk/sciebo/prefect_prj/data/Sentinel_images'

water_source_file = os.path.join(base,"joined-surveys-2013-grouped.csv")
corr_GE_HR_file = os.path.join(base, "corresponding_ge_hr_survey.csv")
batch_size = 20
# 3 Main label categories (which are kept)
main_labels = ['piped', 'groundwater', 'bottled water']
num_labels = len(main_labels)
#Training, validation and test set size 
#[Training size, validation size, test size]
split_size = [0.8,0.1,0.1]
#Input height
input_height  = 200
#Input width 
input_width = 200
#Minimum and maximum values (for clipping above and below those values)
#[Minimum value, maximum value]
clipping_values = [0, 3000]

#channels (define channels which should be used, if all should list can stay empty channel = [])
channels = [4,3,2]

## Preprocess part

Run the following part only once at the beginning if needed.

In [ ]:
#Function : To create your urban directory with all image belonging to the urban type in it
urban_dir, channel_size =  get_urban_img(img_dir, base, channels)
print('Moved urbane images to seperate folder')

In [ ]:
#Create Label Data Frame (Saved after splitting them up)
urban_dir ='/mnt/datadisk/shannon/NN/urban'
labels_df = get_labels_df_for_img(water_source_file, urban_dir, main_labels, corr_GE_HR_file)
print("Add to label names categorical labels and removed images which don't belong to one of the main labels")

In [ ]:
#Split into training, validation and test sets
train_dir, val_dir, test_dir = created_data_sets(urban_dir, split_size)
print('Split up data set into training, validation and test data')

In [ ]:
#After splitting them to ensure that labels_df.to_csv is not part of one of the three subsets
labels_df.to_csv('/mnt/datadisk/shannon/NN/urban/labels_df.csv')

## 2nd Part : Model

Means and Std Calculation part may take a while (but not as long as the once before); hence, you may save means and std as parameter

In [ ]:
#Defined again if part before is skipped
train_dir = '/mnt/datadisk/shannon/NN/urban/training'
val_dir = '/mnt/datadisk/shannon/NN/urban/validation'

In [ ]:
#Calculate mean and standard deviation, print them such that you could save them to variables if wanted (do not have to 
#to calculate mean & standard deviation again)
means = calc_mean(train_dir, input_height, input_width, clipping_values, channels)
stds = calc_std(means, train_dir, input_height, input_width, clipping_values, channels)
print(means, stds)
print('Calculated mean and standard deviation for each channel (for training set)') 

In [ ]:
#Read in labels_df(if you closed the program in between)
labels_df = pd.read_csv('/mnt/datadisk/shannon/NN/urban/labels_df.csv')
channel_size = 3
labels_df

In [ ]:
#Create class weights
class_weights = create_class_weights_dict(train_dir, labels_df)
print(class_weights)


In [ ]:
#This partonly checks if the normalization worked fine
training_generator = generator(train_dir, labels_df, batch_size, means, stds, input_height, input_width, clipping_values, channels, channel_size, num_labels)
#Check if shape is correct
print('Created x and y for training data')
for data_batch, labels_batch in training_generator:
    print('This is the shape of the training data batch:', data_batch.shape)
    print('This is the shape of the training label batch:', labels_batch.shape)
    samples = data_batch
    break

validation_generator = generator(val_dir, labels_df, batch_size, means, stds, input_height, input_width, clipping_values, channels, channel_size, num_labels)

print('The minimum value is', np.min(samples))
print('The maximum value is', np.max(samples))
print('The mean is', np.mean(samples))
print('The standard deviation is', np.std(samples))

import matplotlib.pyplot as plt

plt.hist(samples.flatten(), bins = 100)
plt.show()

## Neuronal Network part

In [ ]:
#This part generates the actually training generator for the NN
from functools import partial

train_generator_func = partial(generator, train_dir, labels_df, batch_size, means, stds, input_height, input_width, clipping_values, channels, channel_size, num_labels)
train_ds = tf.data.Dataset.from_generator(train_generator_func, 
                                          output_types=(tf.float32, tf.float32), 
                                          output_shapes=((batch_size, 200, 200, 3), (batch_size, 3)),
                                         )

'''for elem in train_ds.take(2):
    print(elem)'''

In [ ]:
#This part generates the actually validation generator for the NN
val_generator_func = partial(generator, val_dir, labels_df, batch_size, means, stds, input_height, input_width, clipping_values, channels, channel_size, num_labels)
val_ds = tf.data.Dataset.from_generator(val_generator_func, 
                                          output_types=(tf.float32, tf.float32), 
                                          output_shapes=((batch_size, 200, 200, 3), (batch_size, 3)),
                                       )

In [ ]:
#RESNET

base_model = tensorflow.keras.applications.ResNet50(
    include_top=True,
    weights = None,
    input_shape = (200, 200, channel_size),
    pooling = None,
    classes = 3,
    
)
base_model.summary()

In [ ]:
opt = optimizers.Adam(learning_rate=1e-3)

base_model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

history = base_model.fit(
            train_ds,
            class_weight=class_weights,
            validation_data=val_ds,
            epochs=100)